In [21]:
import pandas as pd
import os

import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

SOURCE_PATH = os.path.dirname(os.getcwd())

## Load and preprocess data

In [42]:
data = pd.read_csv(os.path.join(SOURCE_PATH, "data", "inputs", "ar1_150_random", "betadgp_beta2x2_data.csv"))
y = data["betas_dgp"].to_numpy()
X = data[["cov_dgp"]].to_numpy()

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.7)

In [43]:
X_train.shape

(15749, 1)

In [44]:
X_validation.shape

(6751, 1)

In [47]:
scaler = StandardScaler()
X_train_zscore = scaler.fit_transform(X_train)
X_validation_zscore = scaler.transform(X_validation)